In [1]:
HOST = "10.128.0.6"
PORT = "27017"
DB = "final_project"

In [2]:
from pymongo import MongoClient

def connect():
    url = "mongodb://{}:{}".format(HOST, PORT)
    connection = MongoClient(url)
    print(connection)
    return connection[DB]

## 1. Test connection

In [3]:
connection = connect()

print(connection)

MongoClient(host=['10.128.0.6:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['10.128.0.6:27017'], document_class=dict, tz_aware=False, connect=True), 'final_project')


## 2. Write records into the collection

In [4]:
# This is from the sample code to test the connection

from pymongo import MongoClient

rec1 = {
  "first_name" : "James",
  "last_name" : "Butt Benton",
  "company" : "John B Jr",
  "address" : "6649 N Blue Gum St", 
  "city" : "New Orleans", 
  "county" : "Orleans", 
  "state" : "LA", 
  "zip" : "70116",
  "phone_1" : "504-621-8927",
  "phone_2" : "504-845-1427",
  "email" : "jbutt@gmail.com", 
  "web" : "http://www.bentonjohnbjr.com" 
}

rec2 = {
  "first_name" : "Josephine",
  "last_name" : "Darakjy Chanay",
  "company" : "Jeffrey A Esq",
  "address" : "4 B Blue Ridge Blvd", 
  "city" : "Brighton", 
  "county" : "Livingston", 
  "state" : "MI", 
  "zip" : "48116",
  "phone_1" : "810-292-9388",
  "phone_2" : "810-374-9840",
  "email" : "josephine_darakjy@darakjy.org", 
  "web" : "http://www.chanayjeffreyaesq.com" 
}

rec3 = {
  "first_name" : "Art",
  "last_name" : "Venere",
  "company" : "Chemel, James L Cpa",
  "address" : "8 W Cerritos Ave #54", 
  "city" : "Bridgeport", 
  "county" : "Gloucester", 
  "state" : "NJ", 
  "zip" : "08014",
  "phone_1" : "856-636-8749",
  "phone_2" : "856-264-4130",
  "email" : "art@venere.org", 
  "web" : "http://www.chemeljameslcpa.com" 
}

try:
    connection = connect()
    collection = connection["person"]
    collection.insert_many([rec1, rec2, rec3])
    print("Wrote 3 documents into collection")
except Exception as e:
    print("Error occurred while writing documents:", e)

MongoClient(host=['10.128.0.6:27017'], document_class=dict, tz_aware=False, connect=True)
Wrote 3 documents into collection


## Part 3. MongoDB

In [5]:
!pip install faker_airtravel
!pip install faker

### Checking the fake.Flight() dict of keys and values

In [6]:
from faker import Faker
from faker_airtravel import AirTravelProvider

# Create Faker object and add the AirTravelProvider
fake = Faker()
fake.add_provider(AirTravelProvider)

# Use the fake object to generate flight information
flight_info = fake.flight()

print(flight_info)

{'airline': 'China Eastern Airlines', 'origin': {'airport': 'Belize city Municipal airport', 'iata': 'TZA', 'icao': '', 'city': 'Hattieville', 'state': 'Belize', 'country': 'Belize'}, 'destination': {'airport': 'Winnipeg International airport', 'iata': 'YWG', 'icao': 'CYWG', 'city': 'Winnipeg', 'state': 'Manitoba', 'country': 'Canada'}, 'stops': 'non-stop', 'price': 396}


### Populate the ticketing collection with 100 unique records

In [7]:
# Import libraries and packages
from pymongo import MongoClient
from faker import Faker
from faker_airtravel import AirTravelProvider
from datetime import datetime, timedelta
import random

# Create a Faker instance and add the AirTravelProvider for air travel-related dat
fake = Faker()
fake.add_provider(AirTravelProvider)
Faker.seed(30)  # Setting seed for consistent data generation

# Function to generate random date within a range
def random_date(start_date, end_date):
    return fake.date_time_between(start_date=start_date, end_date=end_date)

# Function to generate random time within a range
def random_time():
    return timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59), seconds=random.randint(0, 59))

# Function to generate data for MongoDB ticketing collection
def generate_ticketing_data():
    ticketing_data = []
    for _ in range(100):
        dep_date = random_date('-30d', '+30d').date()
        arr_date = random_date('-30d', '+30d').date()
        
        # Set the time part of the date fields to '00:00:00'
        dep_datetime = datetime.combine(dep_date, datetime.min.time())
        arr_datetime = datetime.combine(arr_date, datetime.min.time())
        
        # Generate flight information using fake.flight()
        flight_info = fake.flight()
        
        record = {
            'tck_id': str(fake.uuid4()),
            'cust_id': str(fake.uuid4()),
            'airline': flight_info['airline'],
            'flight_nm': fake.random_int(1000, 9999),
            'dep_airport': flight_info['origin']['airport'],
            'arr_airport': flight_info['destination']['airport'],
            'dep_date': dep_datetime,
            'dep_time': dep_datetime + random_time(),
            'arr_date': arr_datetime,
            'arr_time': arr_datetime + random_time(),
            'stops': flight_info['stops'],
            'tik_amt': round(flight_info['price'], 2),
            'curr_code': fake.currency_code()
        }
        ticketing_data.append(record)
    return ticketing_data

# Use generate_ticketing_data() function to populate your MongoDB collection
ticketing_collection = generate_ticketing_data()

# Insert the generated data into MongoDB collection named 'ticketing'
try:
    connection = connect()
    collection = connection["ticketing"]
    collection.insert_many(ticketing_collection)
    print("100 documents written into ticketing collection") # Success message if insertion is successful
except Exception as e:
    print("Error occurred while writing documents:", e) # Error message

MongoClient(host=['10.128.0.6:27017'], document_class=dict, tz_aware=False, connect=True)
100 documents written into ticketing collection
